In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [6]:
def read_file(filename):
    mapping=[]
    directions=None
    get_dir=False
    with open(filename,'r') as f:
        for line in f.readlines():
            if not get_dir:
                line = line.strip('\n')
                if len(line) > 0:
                    mapping.append(list(line))
                else: 
                    get_dir=True
                    continue
            else:
                directions=line.strip()
                
    return mapping, directions

In [27]:
def get_init_pos(mp):
    max_row=len(mp)
    i=0
    while mp[-1][i] != '.':
        i+=1
    return (max_row-1, i)
    

In [40]:
def read_directions(direction):
    rotate=[]
    turns =[]
    for i,c in enumerate(d):
        if not c.isdigit():
            rotate.append(i)
            turns.append(-1 if c == 'L' else 1)
        
    steps=[]
    for s,e  in zip([-1]+rotate, rotate+[len(d)]):
        steps.append(int(d[s+1:e]))
    return steps,turns

In [74]:
def move_position(steps, bearing, mapping, pos):
    max_row=len(mapping)
    max_col=len(mapping[1])
    rc, cc = bearing
    ir,ic = pos
    wrap=0
    new_row=None
    new_col=None
    
    for i in range(1,steps+1):
        new_row=(ir+rc*(i+wrap))%max_row
        new_col=(ic+cc*(i+wrap))%max_col
        print(new_row,new_col,max_col)
        if mapping[new_row][new_col] == '#':
            ### Problem with the wrapping on this
            new_row=(ir+rc*(i+wrap-1))%max_row
            new_col=(ic+cc*(i+wrap-1))%max_col
            pos = [new_row,new_col]
            return pos
        
        if mapping[new_row][new_col] == '':
            while mapping[new_row][new_col] == '':
                wrap+=1
                new_row=(ir+rc*(i+wrap))%max_row
                new_col=(ic+cc*(i+wrap))%max_col
        
    pos = [new_row,new_col]
    return pos

In [75]:
from collections import deque

bearing=[ (0,1),
          (-1,0),
          (0,-1),
          (1,0),
        ]

def part_one(filename):
    mp, d = read_file(filename)
    mp.reverse()
    
    steps, turns = read_directions(d)
    pos = get_init_pos(mp)
    b=0
   
    p = 0
    t = 0
    while p < len(turns):
        pos = move_position(steps[p], bearing[b], mp, pos)
        b = (b + turns[t])%4
        p+=1
        t+=1
    
    pos = move_position(steps[-1], bearing[b], mp, pos)
    
    return (len(mp)-pos[0])*1000 + 4*(pos[1]+1) + b
        
    

In [76]:
%%time
part_one('input/test_22.txt')

11 9 16
11 10 16
11 11 16
10 10 16
9 10 16
8 10 16
7 10 16
6 10 16
6 11 16
6 12 16


IndexError: list index out of range

In [73]:
%%time
part_one('input/day_22.txt')

199 1 50
199 2 50
199 3 50
199 4 50
199 5 50
199 6 50
199 7 50
199 8 50
199 9 50
199 10 50
198 10 50
197 10 50
196 10 50
195 10 50
194 10 50
194 11 50
194 12 50
194 13 50
194 14 50
194 15 50
193 15 50
192 15 50
191 15 50
190 15 50
189 15 50
188 15 50
187 15 50
186 15 50
185 15 50
184 15 50
184 16 50
184 17 50
184 18 50
184 19 50
183 19 50
182 19 50
181 19 50
180 19 50
179 19 50
179 20 50
179 21 50
179 22 50
179 23 50
179 24 50
CPU times: user 5.04 ms, sys: 4.04 ms, total: 9.07 ms
Wall time: 7.13 ms


21100

In [66]:
# 9264 is too low

# Part 2